In [1]:
from bs4 import BeautifulSoup
from pprint import pprint
import requests
import urllib.request
import os
import urllib.parse
import shutil
#import cv2
from PIL import Image

import tensorflow.keras
from PIL import Image, ImageOps
import numpy as np


In [2]:
fakeBlogKeywordList = ["스토리앤","seoulouba","mateb.kr","revu","weble","ohmyblog","mrblog","tble",
    "dinnerqueen","%EA%B3%B5%EC%A0%95%EA%B1%B0%EB%9E%98%EC%9C%84%EC%9B%90%ED%9A%8C-%EB%AC%B8%EA%B5%AC",
    "%EB%B8%94%EB%A1%9C%EA%B7%B8%EC%9B%90%EC%A0%95%EB%8C%80",
    "banner_","%EC%9D%B4%EC%8A%88%EB%B8%94%EB%A1%9C%EA%B7%B8",
    "%EB%B0%B0%EB%84%88","http://echoblog.net/images/sponsor-banner.png",
                      "sponsor","banner","echoblog"]

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

#define const value
batch_size = 3000
epochs = 15
IMG_HEIGHT = 260
IMG_WIDTH = 460


In [4]:
def create_model():
    model = Sequential([
        Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
        MaxPooling2D(),
        Conv2D(32, 3, padding='same', activation='relu'),
        MaxPooling2D(),
        Conv2D(64, 3, padding='same', activation='relu'),
        MaxPooling2D(),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
    return model

In [5]:
#load model
checkpoint_path = "model_diff_width_0624_v1(batch-3000)/cp-0015.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)

# create model
model = create_model()
model.load_weights(latest)

data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
teachable_model = tensorflow.keras.models.load_model('/Users/esens/Downloads/converted_keras/keras_model.h5')
np.set_printoptions(suppress=True)

model_diff_width_0624_v1(batch-3000)/cp-0015.ckpt


In [11]:

# =====================================================================
#이미지 분류 [teachable machine model]
def teachableMachinePrediction(image_path):
    try:
        image = Image.open(image_path)
        size = (224, 224)
        image = ImageOps.fit(image, size, Image.ANTIALIAS)
        resize = Image.open(image_path);
        print("image size : ",resize.size);
        
        image_array = np.asarray(image)
        normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
        newdata = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)    
        
#         data = np.array([np.array(cv2.imread(image_path[i])) for i in range(len(image_path))])
#         features = data.flatten().reshape(1, 224,224,3)
#         print("features : " , features.shape)

        if(normalized_image_array.shape == (224,224,4)):
            print('224,224,4 : ' , image_path)
            print(normalized_image_array.shape)
            normalized_image_array_reshape = normalized_image_array.reshape(224,224,3)
            print(normalized_image_array_reshape.shape)

        elif (normalized_image_array.shape == (224, 224)):
            print('224,224 : ', image_path)
            normalized_image_array_reshape = normalized_image_array.reshape(-1,224,224,3)
            print(normalized_image_array_reshape.shape)

        else:
            normalized_image_array_reshape = normalized_image_array.reshape(1,224,224,3)

        # Load the image into the array
        newdata[0] = normalized_image_array_reshape
        #todo image shape 다른경우 처리해줘야함(png일 때 로 추측됨..)
        print('newdata.shape : ' , newdata.shape)
  
        #predictions class 
        # {0 : advertise , 1: others}
        
        prediction = teachable_model.predict(newdata)
        prediction_class = teachable_model.predict_classes(newdata)
        #print("========pridiction[teachable]========")
        #print(prediction_class)
    except Exception as ex:
        print("predict error[teachable machine] : " , ex)
        
    return prediction_class

    
# =====================================================================
#이미지 분류 [my model]

def imageClassification(image_path):
    image = tf.keras.preprocessing.image.load_img(image_path,target_size=(IMG_HEIGHT, IMG_WIDTH))
    input_arr = tf.keras.preprocessing.image.img_to_array(image)
    print("input_arr type : " ,type(input_arr))
    input_arr = np.array([input_arr])  # Convert single image to a batch.

    predictions_class = model.predict_classes(input_arr)
    predictions = model.predict(input_arr)

    print("========pridiction========");
#     print(test_data_gen.class_indices)
    print("predictions class : " , predictions_class);
    print("predictions : " , predictions);
#    print("predictions argmax : ",np.argmax(predictions[0])); # same way
    return predictions_class;

# =====================================================================
#이미지 저장
def saveImage(div_option,index,img_url,nickName,postNumber):
    #블로그 유저 별로 폴더를 나눌건지에 대한 옵션
    #true -> path : image/{nickName}/{postNumber}
    #false -> path : image/myimg/
    if(div_option):
        if nickName is "":
            nickName = "unknown"
        if postNumber is "":
            postNumber = "1"        
        imgPath = f'image/{nickName}/{postNumber}/{index}.jpg'
        dirPath = f'image/{nickName}/{postNumber}'
    #폴더별로 분기하지 않을떄 path : image/myimg/
    else:
        list = os.listdir("image/temp")
        last_index = len(list)
#         print("last_index : ",last_index)
        dirPath = 'image/temp'
        imgPath = f'image/temp/{last_index}.jpg'
    
    if not os.path.isdir(dirPath):
        #폴더 생성
        try:
            os.makedirs(dirPath)
        except Exception as ex:
            print("error catch : " , ex)
                
    #파일 저장
    #temp에 저장 -> classification -> 분류 저장
    
    try:
        filename = img_url.split('/')[-1].split('?')[0];
        img_url2 = urllib.parse.quote_plus(str(filename))
        img_url_final = img_url.replace(img_url.split('/')[-1].split('?')[0],img_url2)
        print("img_url_final : " ,img_url_final);
        for item in fakeBlogKeywordList:
            if(img_url_final.find(item) > 0):
                print('this is fake img')

                imgPath = f"image/temp/fake_{last_index}.jpg"
                break;
        
        urllib.request.urlretrieve(img_url_final, imgPath)
        #분류
        #{'advertise': 0, 'image': 1}
        my_prediction  = imageClassification(imgPath);
        #my_prediction  = teachableMachinePrediction(imgPath);
#         print("prediction : ", my_prediction[0]);
        
        if (my_prediction[0] == 0):
            list = os.listdir("ad_img")
            last_index = len(list)
            #os.rename(imgPath, f'ad_img/{last_index}.jpg')
            #shutil.move(imgPath, f'ad_img/{last_index}.jpg')
            os.replace(imgPath, f'ad_img/{last_index}.jpg')
            return False
        return True
    except Exception as ex:
        print("img save error : " , ex)
        return False
    
    
# =====================================================================
# image gethering
def findBlogImgGethering(searchQuery, searchOption, page):
    url = f'https://search.naver.com/search.naver?query={searchQuery}&sm=tab_pge&srchby=all&st={searchOption}&where=post&start={page}'
    html = requests.get(url)
    # 1차, blog URL 찾기
    soup = BeautifulSoup(html.text, 'html.parser')
    index = 0;

    url_list = [];
    for li_item in soup.find_all('li',{'class' : 'sh_blog_top'}):
        child_item = li_item.find('a',{'class' : 'sh_blog_title'})
        title = child_item.attrs['title']
        href = child_item.attrs['href']

        #url list 저장
        url_list.append(href)

        print(f'title : {title} , href : {href}')
        print('----------------------------------')
        
    # url 파싱 및 예외처리
    for url_item in url_list:
        nickName = "";
        postNumber = "";

        if(url_item.find("blog.me") > 0):
            parsing = url_item;
            parsing = parsing.replace("https://","")
            nickName = parsing.split('.')[0]
            postNumber = parsing.split('/')[1]
            blogUrl = "https://m.blog.naver.com/" + nickName + "?Redirect=Log&logNo=" + postNumber

        else:
            blogUrl = url_item.replace("https://", "https://m.");

        nickName = blogUrl.split('/')[-1].split('?')[0]
        postNumber = blogUrl.split('/')[-1].split('?')[1].split('=')[-1]

        if nickName is "":
            nickName = "unknown"
        if postNumber is "":
            postNumber = "1"
            
        print("------ digging more -------")
        print("nickName : " , blogUrl.split('/')[-1].split('?')[0])
        print("postNumber : " , blogUrl.split('/')[-1].split('?')[1].split('=')[-1])
        print("blogUrl : " , blogUrl)        

        #전체 목록 순회
        blog_html = requests.get(blogUrl)
        blog_soup = BeautifulSoup(blog_html.text, 'html.parser')
        blog_image_class = blog_soup.find_all('div',{'class' : 'se-image'});

        for div_obj in blog_image_class:
            for idx, img_item in enumerate(div_obj.find_all('img')):
                img_url = img_item.attrs['src'];

                if(img_url.find("w80_blur") > 0):
                    img_url = img_url.replace("w80_blur" , "w800")
                isSaveSuccess = saveImage(False,index,img_url,nickName,postNumber)
                if(isSaveSuccess):  
                    index = index+1



In [12]:
searchQuery = "종각 맛집"
searchOption = 'sim'  #sim or date  
page = 1

findBlogImgGethering(searchQuery,searchOption,page)

title : 기가 막혔던 종각 맛집 , href : https://forurwinter.blog.me/221988900945
----------------------------------
title : 푸짐했던 종각 맛집 , href : https://blog.naver.com/celinicious?Redirect=Log&logNo=222006262433
----------------------------------
title : 짜릿했던 종각 맛집 , href : https://blog.naver.com/maeng_ee?Redirect=Log&logNo=221977654830
----------------------------------
title : 가성비 쩔었던 종각 맛집, 교대이층집 , href : https://blog.naver.com/el512?Redirect=Log&logNo=222008681533
----------------------------------
title : 끝장난 종각 맛집 , href : https://ssomerry.blog.me/221983614903
----------------------------------
title : 믿고갔던 종각 맛집 , href : https://blog.naver.com/porky0122?Redirect=Log&logNo=221991050525
----------------------------------
title : 0324 종각 고기집/콜키지프리 맛집 고메식당 , href : https://blog.naver.com/p_radn?Redirect=Log&logNo=221878088433
----------------------------------
title : 까리했던 종각 맛집 , href : https://blog.naver.com/ju_love1202?Redirect=Log&logNo=221952793333
----------------------------------
title

========pridiction========
predictions class :  [[1]]
predictions :  [[2041.5247]]
img_url_final :  https://mblogthumb-phinf.pstatic.net/MjAyMDA2MDNfMTkw/MDAxNTkxMTc3NzMzOTUz.8Qb9xJrLh6iJPw2jY2M3UxAbx1PrY5AE32drET_J9yUg.f4-4A2aXdcbFRuKogDCRVM-NCljtkPVzN7IMTurIxYgg.JPEG.forurwinter/22.JPG?type=w800
input_arr type :  <class 'numpy.ndarray'>
========pridiction========
predictions class :  [[1]]
predictions :  [[1562.0582]]
img_url_final :  https://mblogthumb-phinf.pstatic.net/MjAyMDA2MDNfMTc4/MDAxNTkxMTc3NzM0MDcw.yOmfTb4d1LfB7SEhQ16PR1ZG13DfDqOWZKd-5c096v4g.FXjgnkadmppJoluIleGC8thbXZpwQafvKz_ksZQl4s8g.JPEG.forurwinter/23.JPG?type=w800
input_arr type :  <class 'numpy.ndarray'>
========pridiction========
predictions class :  [[1]]
predictions :  [[2641.2659]]
img_url_final :  https://mblogthumb-phinf.pstatic.net/MjAyMDA2MDNfMTk4/MDAxNTkxMTc3NzM0MTMy.bW1p4QDqfufeXfBjlTIn1LT9cibZFJEp9kRWMnDMkR8g.X7dv0oR70erncKFxFjT9ogjeyJOnDLcsfvBL0I8NmOEg.JPEG.forurwinter/24.JPG?type=w800
input_arr type :  <

KeyboardInterrupt: 